<a href="https://colab.research.google.com/github/anshupandey/ms-generativeai-apr2025/blob/main/code13_tool_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tool calling


## Overview

Many AI applications interact directly with humans. In these cases, it is appropriate for models to respond in natural language.
But what about cases where we want a model to also interact *directly* with systems, such as databases or an API?
These systems often have a particular input schema; for example, APIs frequently have a required payload structure.
This need motivates the concept of *tool calling*.

You can use [tool calling](https://platform.openai.com/docs/guides/function-calling/example-use-cases) to request model responses that match a particular schema.

:::info
You will sometimes hear the term `function calling`. We use this term interchangeably with `tool calling`.
:::

![Conceptual overview of tool calling](https://python.langchain.com/assets/images/tool_calling_concept-552a73031228ff9144c7d59f26dedbbf.png)



## Key concepts

**(1) Tool Creation:** Use the [@tool](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.convert.tool.html) decorator to create a [tool](/docs/concepts/tools). A tool is an association between a function and its schema.
**(2) Tool Binding:** The tool needs to be connected to a model that supports tool calling. This gives the model awareness of the tool and the associated input schema required by the tool.
**(3) Tool Calling:** When appropriate, the model can decide to call a tool and ensure its response conforms to the tool's input schema.
**(4) Tool Execution:** The tool can be executed using the arguments provided by the model.

![Conceptual parts of tool calling](https://python.langchain.com/assets/images/tool_calling_components-bef9d2bcb9d3706c2fe58b57bf8ccb60.png)



## Recommended usage

This pseudo-code illustrates the recommended workflow for using tool calling.
Created tools are passed to `.bind_tools()` method as a list.
This model can be called, as usual. If a tool call is made, model's response will contain the tool call arguments.
The tool call arguments can be passed directly to the tool.


```python
# Tool creation
tools = [my_tool]
# Tool binding
model_with_tools = model.bind_tools(tools)
# Tool calling
response = model_with_tools.invoke(user_input)
```

### Custom Tool Creation
The recommended way to create a tool is using the `@tool` decorator.



The **tool** abstraction in LangChain associates a Python **function** with a **schema** that defines the function's **name**, **description** and **expected arguments**.


## Key concepts

- Tools are a way to encapsulate a function and its schema in a way that can be passed to a chat model.
- Create tools using the @tool decorator, which simplifies the process of tool creation, supporting the following:
   - Automatically infer the tool's **name**, **description** and **expected arguments**, while also supporting customization.
   - Defining tools that return **artifacts** (e.g. images, dataframes, etc.)
   - Hiding input arguments from the schema (and hence from the model) using **injected tool arguments**.


The key attributes that correspond to the tool's **schema**:

- **name**: The name of the tool.
- **description**: A description of what the tool does.
- **args**: Property that returns the JSON schema for the tool's arguments.

The key methods to execute the function associated with the **tool**:

- **invoke**: Invokes the tool with the given arguments.
- **ainvoke**: Invokes the tool with the given arguments, asynchronously. Used for async programming with Langchain.

In [1]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.

    This function takes two integer arguments and returns their product.

    Parameters:
    a (int): The first integer to be multiplied.
    b (int): The second integer to be multiplied.

    Returns:
    int: The product of the two integers.

    Example:
    >>> multiply(2, 3)
    6
    >>> multiply(-1, 5)
    -5
    """
    """Multiply a and b."""
    return a * b

In [2]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers.

This function takes two integer arguments and returns their product.

Parameters:
a (int): The first integer to be multiplied.
b (int): The second integer to be multiplied.

Returns:
int: The product of the two integers.

Example:
>>> multiply(2, 3)
6
>>> multiply(-1, 5)
-5
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [3]:
multiply.invoke({"a":3,"b":5})

15

In [4]:
print(multiply.name) # multiply
print(multiply.description) # Multiply two numbers.
print(multiply.args)

multiply
Multiply two integers.

This function takes two integer arguments and returns their product.

Parameters:
a (int): The first integer to be multiplied.
b (int): The second integer to be multiplied.

Returns:
int: The product of the two integers.

Example:
>>> multiply(2, 3)
6
>>> multiply(-1, 5)
-5
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [5]:
@tool
def get_weather(city_name:str):
    """
    This function is used to get the current weather information.
    ARguments: 
    city_name: name of city for which the weather information is to be fetched.
    returns:
    output: A JSON object with keys weather, temperature and unit
    """
    owmkey=os.environ['OWM']
    api = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={owmkey}"

    response = requests.get(api)
    response = response.content.decode()
    response = json.loads(response)
    output = {"weather":response['weather'][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}
    return output

In [6]:
get_weather.name

'get_weather'

In [7]:
get_weather.description

'This function is used to get the current weather information.\nARguments: \ncity_name: name of city for which the weather information is to be fetched.\nreturns:\noutput: A JSON object with keys weather, temperature and unit'

In [8]:
get_weather.args

{'city_name': {'title': 'City Name', 'type': 'string'}}

## Tool binding


The central concept to understand is that LangChain provides a standardized interface for connecting tools to models.
The `.bind_tools()` method can be used to specify which tools are available for a model to call.


In [9]:
from langchain_openai import ChatOpenAI
import os
model = ChatOpenAI(model='gpt-4o-mini',temperature=0.2)

In [10]:
tool_list=[multiply]

In [11]:
model_with_tools = model.bind_tools(tool_list)

## Tool calling

![Diagram of a tool call by a model](https://python.langchain.com/assets/images/tool_call_example-2348b869f9a5d0d2a45dfbe614c177a4.png)

A key principle of tool calling is that the model decides when to use a tool based on the input's relevance. The model doesn't always need to call a tool.
For example, given an unrelated input, the model would not call the tool:


In [12]:
result = model_with_tools.invoke("Hello world!")
print(result.model_dump_json())

{"content":"Hello! How can I assist you today?","additional_kwargs":{"refusal":null},"response_metadata":{"token_usage":{"completion_tokens":11,"prompt_tokens":118,"total_tokens":129,"completion_tokens_details":{"accepted_prediction_tokens":0,"audio_tokens":0,"reasoning_tokens":0,"rejected_prediction_tokens":0},"prompt_tokens_details":{"audio_tokens":0,"cached_tokens":0}},"model_name":"gpt-4o-mini-2024-07-18","system_fingerprint":"fp_b376dfbbd5","id":"chatcmpl-BIUnpegcWKfayqHt8IOJUAZZD38IW","finish_reason":"stop","logprobs":null},"type":"ai","name":null,"id":"run-e4a718a0-0547-4194-947c-876e2760fe3c-0","example":false,"tool_calls":[],"invalid_tool_calls":[],"usage_metadata":{"input_tokens":118,"output_tokens":11,"total_tokens":129,"input_token_details":{"audio":0,"cache_read":0},"output_token_details":{"audio":0,"reasoning":0}}}


The result would be an `AIMessage` containing the model's response in natural language (e.g., "Hello!").
However, if we pass an input *relevant to the tool*, the model should choose to call it:



In [13]:
result = model_with_tools.invoke("What is 2 multiplied by 3?")
print(result.model_dump_json(indent=1))

{
 "content": "",
 "additional_kwargs": {
  "tool_calls": [
   {
    "id": "call_S2uAnuLIL1ZX1VUrcirUJFuG",
    "function": {
     "arguments": "{\"a\":2,\"b\":3}",
     "name": "multiply"
    },
    "type": "function"
   }
  ],
  "refusal": null
 },
 "response_metadata": {
  "token_usage": {
   "completion_tokens": 18,
   "prompt_tokens": 124,
   "total_tokens": 142,
   "completion_tokens_details": {
    "accepted_prediction_tokens": 0,
    "audio_tokens": 0,
    "reasoning_tokens": 0,
    "rejected_prediction_tokens": 0
   },
   "prompt_tokens_details": {
    "audio_tokens": 0,
    "cached_tokens": 0
   }
  },
  "model_name": "gpt-4o-mini-2024-07-18",
  "system_fingerprint": "fp_b376dfbbd5",
  "id": "chatcmpl-BIUo3uHXemNaturxt29UfBW9Z9zRY",
  "finish_reason": "tool_calls",
  "logprobs": null
 },
 "type": "ai",
 "name": null,
 "id": "run-d7f3f18b-bdbd-4336-81db-7d7a0922d146-0",
 "example": false,
 "tool_calls": [
  {
   "name": "multiply",
   "args": {
    "a": 2,
    "b": 3
   },
   

As before, the output `result` will be an `AIMessage`.
But, if the tool was called, `result` will have a `tool_calls` attribute.
This attribute includes everything needed to execute the tool, including the tool name and input arguments:


In [14]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_S2uAnuLIL1ZX1VUrcirUJFuG',
  'type': 'tool_call'}]


## Best practices

When designing [tools](/docs/concepts/tools/) to be used by a model, it is important to keep in mind that:

* Models that have explicit [tool-calling APIs](/docs/concepts/tool_calling) will be better at tool calling than non-fine-tuned models.
* Models will perform better if the tools have well-chosen names and descriptions.
* Simple, narrowly scoped tools are easier for models to use than complex tools.
* Asking the model to select from a large list of tools poses challenges for the model.




# Thank You